In [1]:
from brainlit.map_neurons.map_neurons import (
    DiffeomorphismAction,
    transform_geometricgraph,
    compute_derivs,
    CloudReg_Transform,
)
from brainlit.algorithms.trace_analysis.fit_spline import (
    GeometricGraph,
    compute_parameterization,
    CubicHermiteChain
)
from brainlit.utils.Neuron_trace import NeuronTrace

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.interpolate import splev, splprep, CubicHermiteSpline
from scipy.spatial.distance import cosine
from scipy.stats import wilcoxon, pearsonr, linregress, norm
from tqdm import tqdm
from cloudvolume import CloudVolume
from similaritymeasures import frechet_dist
import seaborn as sns
from statannotations.Annotator import Annotator
from statannot import add_stat_annotation
from cloudvolume.exceptions import SkeletonDecodeError
import pickle
from copy import deepcopy
import networkx as nx
import os

In [2]:
root_dir = Path(os.path.abspath(""))
data_dir = os.path.join(root_dir, "data", "mapping-files")

In [3]:
velocity_path = os.path.join(data_dir, "downloop_1_v.mat")
affine_path = os.path.join(data_dir, "downloop_1_A.mat")

ct = CloudReg_Transform(velocity_path, affine_path)

integrating velocity field: 100%|██████████| 10/10 [00:23<00:00,  2.33s/it]


In [4]:
axons_path = "precomputed://file://" + os.path.join(data_dir, "axons")
vol = CloudVolume(axons_path)
shp = np.array(vol.shape)
res = np.array(vol.resolution) / 1000
origin = (shp[:3] - 1) * res / 2

skel = vol.skeleton.get(4)
coords = skel.vertices / 1000 - origin
coords = ct.apply_affine(coords)

## original vertices

In [5]:
print(skel.vertices)

[[6680812.5 4213735.5 5570578. ]
 [6679242.  4213755.  5572043. ]
 [6660847.5 4216505.  5564594. ]
 ...
 [7051972.5 3908219.8 4851556.5]
 [7074937.5 4341430.5 5281373. ]
 [7068199.  4344958.  5270404.5]]


## orient

In [6]:
coords = skel.vertices / 1000 - origin
print(coords)

[[1468.0125      229.73535156  482.578125  ]
 [1466.4421875   229.75488281  484.04296875]
 [1448.04765625  232.50488281  476.59423828]
 ...
 [1839.17265625  -75.78027344 -236.44335938]
 [1862.1375      357.43066406  193.37304688]
 [1855.39921875  360.95800781  182.40429688]]


## affine

In [7]:
coords = ct.apply_affine(coords)
print(coords)

[[ 524.30916956  737.44099749 1371.16397248]
 [ 525.84386745  737.47461986 1369.48174897]
 [ 518.37428854  739.81165879 1349.2803704 ]
 ...
 [-210.42621759  335.60447696 1773.2164223 ]
 [ 214.64419038  898.37724199 1790.00024407]
 [ 203.32006118  901.97601536 1782.37493956]]


## complete transform

In [8]:
coords = ct.evaluate(coords)
print(coords)

[[ 528.58474024  736.84231508 1368.12984341]
 [ 530.12180605  736.87501546 1366.44977923]
 [ 522.66172355  739.19192612 1346.27765454]
 ...
 [-206.73472768  336.05556287 1770.35923739]
 [ 218.37173767  897.91184161 1786.17772276]
 [ 207.04544281  901.49831338 1778.55821141]]
